In [1]:
import csv
import ast
import numpy as np
import time
import random
import math

start = time.time()

# --- Load BB (Billboard data) ---
BB = []
with open("BB_NYC.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        BB.append({'idx': int(row['B_index']), 'cost': float(row['Cost'])})

# --- Load ub (User influence data) ---

ub = []
with open("new_ub.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        row['User ID'] = int(row['UserId'])
        # Parse the string dictionary into an actual Python dictionary
        row['Influenced Billboards'] = eval(row['Influenced Billboards'])  # safer: use ast.literal_eval if possible
        ub.append(row)

# --- Load product to user interest mapping ---
product_user_map = []
with open("Product_user_map_NYC.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        row['Venue Category'] = row['Venue Category']

        # Example: "0,1,3" -> [0, 1, 3]
        interested_users = [int(x.strip()) for x in row['User IDs'].split(',') if x.strip().isdigit()]
        row['User IDs'] = interested_users
        product_user_map.append(row)

# --- Load Dem (Demand values) ---
Dem = []
with open("advertiser_1.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        Dem.append(float(row['Demand']))

# --- Load Dem (Demand values) ---
Budget = []
with open("advertiser_1.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        Budget.append(float(row['Payment']))
# print(Budget)
# Convert BB to billboard slots as dictionaries
BS = BB[:5000]
# print(BS)

In [2]:
# --- Influence Function ---
def product_specific_influence(slot_subset, product_id):
    """
    slot_subset: list or set of slot IDs (integers)
    product_id: integer, ID of the product for which influence is calculated
    """
    # Get the users interested in this product
    interested_users = next((row['User IDs'] for row in product_user_map if int(row['Product ID']) == product_id), [])
    # print("Interested users",interested_users)
    influence_score = 0.0
    for user_idx in interested_users:
        # print("user_idx",user_idx)
        user_data = ub[user_idx-1]
        # print("user_data",user_data)
        influenced_billboards = user_data['Influenced Billboards']
        # print("Influenced Billboards",influenced_billboards)
        # Product over (1 - influence probability of selected slots)
        product_term = 1.0
        for slot in slot_subset:
            prob = influenced_billboards.get(slot, 0.0)
            # print("Probability",prob)
            product_term *= (1 - prob)
            # print("Product term",product_term)
        influence_score += (1 - product_term)

    return influence_score

In [3]:
def randomized_allocation(BS, Dem, sigma, iterations):
    l = len(Dem)
    M = dict()
    Flag = dict()
    
    seen_permutations = set()  # To keep track of used (p, q)

    iter = 0
    actual_iters = 0  # To keep track of successful iterations

    while actual_iters < iterations:
        print(f"\n========= Iteration {actual_iters} ======")
        
        # Generate random permutations
        p = tuple(random.sample(range(l), l))               # Advertiser order as tuple
        q = tuple(slot['idx'] for slot in random.sample(BS, len(BS)))  # Billboard IDs order as tuple

        if (p, q) in seen_permutations:
            continue  # Skip this iteration if permutation is already used
        seen_permutations.add((p, q))  # Mark as used
        # print("Seen Permuations",seen_permutations)
        S = set()
        Si_list = [set() for _ in range(l)]         
        vi = [Bgt for Bgt in Budget]   
        # print("vi",vi)
        p_q_key = actual_iters
        M[p_q_key] = None
        Flag[p_q_key] = 1                   

        for i in p:
            # print(f"\n========= For Advertizer{i} ======")
            while vi[i] > 0 and product_specific_influence(Si_list[i],i) < sigma[i]:
                best_slot = None
                best_gain = 0

                for slot in BS:
                    if slot['idx'] in S or slot['idx'] in Si_list[i]:
                        continue
                    if slot['cost'] > vi[i]:
                        continue

                    gain = product_specific_influence(Si_list[i] | {slot['idx']},i) - product_specific_influence(Si_list[i],i)
                    if gain > best_gain:
                        best_gain = gain
                        best_slot = slot

                if best_slot is None:
                    break

                Si_list[i].add(best_slot['idx'])  
                S.add(best_slot['idx'])
                vi[i] -= best_slot['cost']
            if_satisfied = product_specific_influence(Si_list[i],i)
            # print("Influence :",if_satisfied)
            if if_satisfied < sigma[i]:
                print(f"Advertiser {i} demand not satisfied (required: {sigma[i]}, got: {product_specific_influence(Si_list[i],i)})")
                Flag[p_q_key] = 0
                break

        if Flag[p_q_key] == 0:
            M[p_q_key] = "Not Feasible"
        else:
            total_cost = sum(slot['cost'] for slot in BS if slot['idx'] in S)
            M[p_q_key] = (Si_list, total_cost)

        actual_iters += 1
        iter += 1

    # Final selection
    best_solution = None
    min_total_cost = float('inf')
    # print("M.items()",M.items())
    for key, value in M.items():
        if value != "Not Feasible":
            Si_list, total_cost = value
            if total_cost < min_total_cost:
                min_total_cost = total_cost
                best_solution = Si_list

    
    return best_solution, min_total_cost

In [ ]:
# Define thresholds for each advertiser (can be scaled if needed)
sigma = [demand for demand in Dem]

# Number of random iterations
n = 100

# Run the randomized approximation algorithm
final_solution, total_cost = randomized_allocation(BS, Dem, sigma, n)

# Print the final result
if final_solution is not None:
    print("\nFeasible solution found")
    print("Total Cost:", total_cost)
else:
    print("No feasible solution found.")

print("Execution Time:", time.time() - start, "seconds")



========= Iteration 0 ======


In [ ]:
total_selected_slots = 0
total_influence = 0.0

for i, s in enumerate(final_solution):
    sorted_slots = sorted(list(s))                     # Slot indices for advertiser i
    influence_value = product_specific_influence(s, i) # Influence of advertiser i's selected slots
    demand_value = sigma[i]                            # Demand for advertiser i

    total_selected_slots += len(s)                     # Count slots
    total_influence += influence_value                 # Sum influence

    print(f"Advertiser {i+1}: Slots = {sorted_slots}, Demand = {demand_value:.4f}, Influence = {influence_value:.4f}")

# Final summary
print(f"\nTotal number of slots selected by all advertisers: {total_selected_slots}")
print(f"Total influence gain across all advertisers: {total_influence:.4f}")
